<a href="https://colab.research.google.com/github/godey4me/dea_repo/blob/main/1_30_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

# ==========================================
# 0. SETUP: GENERATE RELATIONAL DATA
# ==========================================
# We need two tables to demonstrate Merging (SQL Joins)
np.random.seed(42)

# Table 1: Customers (Dimension Table)
customers = pd.DataFrame({
    'CustomerID': range(101, 106),
    'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve'],
    'Region': ['North', 'North', 'South', 'South', 'West'],
    'Segment': ['Premium', 'Standard', 'Standard', 'Premium', 'Standard']
})

# Table 2: Orders (Fact Table)
# Note: Customer 106 exists here but not in Customers table (to show join differences)
orders = pd.DataFrame({
    'OrderID': range(1001, 1011),
    'CustomerID': np.random.choice([101, 102, 103, 104, 105, 106], 10),
    'Amount': np.random.randint(50, 1000, 10),
    'Status': np.random.choice(['Shipped', 'Pending', 'Cancelled'], 10)
})

print("--- CUSTOMERS TABLE ---")
print(customers.head())
print("\n--- ORDERS TABLE ---")
print(orders.head())
print("-" * 50)

--- CUSTOMERS TABLE ---
   CustomerID     Name Region   Segment
0         101    Alice  North   Premium
1         102      Bob  North  Standard
2         103  Charlie  South  Standard
3         104    David  South   Premium
4         105      Eve   West  Standard

--- ORDERS TABLE ---
   OrderID  CustomerID  Amount   Status
0     1001         104     149  Pending
1     1002         105     921  Pending
2     1003         103     713  Shipped
3     1004         105     180  Shipped
4     1005         105     711  Pending
--------------------------------------------------


In [ ]:
# ==========================================
# 1. MERGING (The "VLOOKUP" of Pandas)
# ==========================================

# A. INNER JOIN (The Default)
# Only keeps rows where the CustomerID exists in BOTH tables.
# Orders from Customer 106 will be dropped because 106 isn't in the Customers table.
print("\n[MERGE] Inner Join (Matches Only):")
merged_inner = pd.merge(orders, customers, on='CustomerID', how='inner')
print(merged_inner.head())
print(f"Rows: {len(merged_inner)}")

# B. LEFT JOIN (The most common for analysis)
# Keep ALL orders, attach customer info where available.
# If customer missing (106), their name becomes NaN.
print("\n[MERGE] Left Join (Keep all Orders):")
merged_df = pd.merge(orders, customers, on='CustomerID', how='left')
print(merged_df.head(10))


[MERGE] Inner Join (Matches Only):
   OrderID  CustomerID  Amount   Status     Name Region   Segment
0     1001         104     149  Pending    David  South   Premium
1     1002         105     921  Pending      Eve   West  Standard
2     1003         103     713  Shipped  Charlie  South  Standard
3     1004         105     180  Shipped      Eve   West  Standard
4     1005         105     711  Pending      Eve   West  Standard
Rows: 10

[MERGE] Left Join (Keep all Orders):
   OrderID  CustomerID  Amount     Status     Name Region   Segment
0     1001         104     149    Pending    David  South   Premium
1     1002         105     921    Pending      Eve   West  Standard
2     1003         103     713    Shipped  Charlie  South  Standard
3     1004         105     180    Shipped      Eve   West  Standard
4     1005         105     711    Pending      Eve   West  Standard
5     1006         102     358    Pending      Bob  North  Standard
6     1007         103     819    Shipped  Ch

In [ ]:
# ==========================================
# 2. FILTERING (Selecting Rows)
# ==========================================
# We use 'merged_df' for the rest of the demo

# A. Simple Condition
# print("\n[FILTER] Orders over $500:")
# high_value = merged_df[merged_df['Amount'] > 500]
# print(f'high value lenght')
# # high_value = merged_df[merged_df['Amount'] > 500]
# print(f"Rows: {len(high_value)}")
# # print(f"Columns: {} ")
# # print(f'high value head')
# # print(high_value.head())
# # print(f'high value name and amount')
# print(high_value[['Name', 'Amount']])

# B. Multiple Conditions (& = AND, | = OR)
# "Show me Shipped orders from the North region"
print("\n[FILTER] Shipped AND North Region:")
shipped_north = merged_df[
    (merged_df['Status'] == 'Shipped') &
    (merged_df['Region'] == 'North')
]
shipped_south = merged_df[
    (merged_df['Status'] == 'Shipped') &
    (merged_df['Region'] == 'South')
    ]
shipped_south2 = merged_df.query("Status == 'Shipped' & Region =='South' ")
# shipped_south = merged_df.query("Status == 'Shipped' & Region == 'South'")

# print(shipped_north[['OrderID', 'Name', 'Region', 'Status']])
# print(f'shipped southhead')
# print(shipped_south2.head())
# print(shipped_south[['OrderID', 'Name', 'Region', 'Status']])

# C. The .isin() trick (Cleaner than multiple ORs)
# "Show me orders that are either Pending OR Cancelled"
print("\n[FILTER] Using .isin(['Pending', 'Cancelled']):")
issues = merged_df[merged_df['Status'].isin(['Pending', 'Cancelled'])]
print(issues[['OrderID', 'Status']])

# D. String Filtering
# "Find customers whose name starts with 'A'"
print("\n[FILTER] Names starting with 'A':")
starts_a = merged_df[merged_df['Name'].str.startswith('A', na=False)]
print(starts_a[['Name', 'Amount']])


[FILTER] Shipped AND North Region:

[FILTER] Using .isin(['Pending', 'Cancelled']):
   OrderID     Status
0     1001    Pending
1     1002    Pending
4     1005    Pending
5     1006    Pending
9     1010  Cancelled

[FILTER] Names starting with 'A':
Empty DataFrame
Columns: [Name, Amount]
Index: []


In [ ]:
print("\n--- More .query() Examples ---")
print(merged_df.dtypes)
# Example 1: Filter by a numerical range
print("\n[QUERY] Orders with Amount between $200 and $500:")
amount_range = merged_df.query('Amount >= 200 and Amount <= 500')
print(amount_range[['Name', 'Amount', 'Status']])

# Example 2: Filter by a specific value in a categorical column
print("\n[QUERY] Orders from 'Eve':")
eve_orders = merged_df.query('Name == "Eve"')
print(eve_orders[['OrderID', 'Name', 'Amount']])

# Example 3: Filter using 'in' operator for multiple categorical values
print("\n[QUERY] Orders from North or West Region:")
north_west_orders = merged_df.query('Region in ["North", "West"]')
print(north_west_orders[['Name', 'Region', 'Amount']])

# Example 4: Combining numerical and categorical conditions
print("\n[QUERY] Premium customers with orders over $500:")
premium_high_value = merged_df.query('Segment == "Premium" and Amount > 500')
print(premium_high_value[['Name', 'Segment', 'Amount']])


--- More .query() Examples ---
OrderID        int64
CustomerID     int64
Amount         int64
Status        object
Name          object
Region        object
Segment       object
dtype: object

[QUERY] Orders with Amount between $200 and $500:
      Name  Amount     Status
5      Bob     358    Pending
7  Charlie     393    Shipped
9      Eve     463  Cancelled

[QUERY] Orders from 'Eve':
   OrderID Name  Amount
1     1002  Eve     921
3     1004  Eve     180
4     1005  Eve     711
9     1010  Eve     463

[QUERY] Orders from North or West Region:
  Name Region  Amount
1  Eve   West     921
3  Eve   West     180
4  Eve   West     711
5  Bob  North     358
9  Eve   West     463

[QUERY] Premium customers with orders over $500:
Empty DataFrame
Columns: [Name, Segment, Amount]
Index: []


In [ ]:
# ==========================================
# 3. SORTING (Ordering Data)
# ==========================================

# A. Sort by one column (Descending)
print("\n[SORT] Highest Amount first:")
sorted_data = merged_df.sort_values(by='Amount', ascending=False)
print(sorted_data[['Name', 'Amount']].head())

# B. Sort by Multiple Columns
# "Sort by Region first, then by Amount within that Region"
print("\n[SORT] By Region (A-Z), then Amount (High-Low):")
multi_sort = merged_df.sort_values(by=['Region', 'Amount'], ascending=[True, False])
print(multi_sort[['Region', 'Amount', 'Name']])

In [ ]:
# ==========================================
# 4. GROUP BY (Aggregation)
# ==========================================

# A. Simple Grouping
# "What is the Total revenue per Region?"
print("\n[GROUPBY] Total Sales by Region:")
region_sales = merged_df.groupby('Region')['Amount'].sum()
print(region_sales)

# B. Grouping by Multiple Columns
# "Total Sales per Region AND Segment"
print("\n[GROUPBY] Sales by Region AND Segment:")
complex_group = merged_df.groupby(['Region', 'Segment'])['Amount'].sum().reset_index()
print(complex_group)

# C. The .agg() function (Custom Stats)
# "Give me the Total, Average, and Count of orders per Customer"
print("\n[GROUPBY] Custom Aggregations (Sum, Mean, Count):")
cust_stats = merged_df.groupby('Name')['Amount'].agg(['sum', 'mean', 'count'])

# Rename columns for cleaner output
cust_stats = cust_stats.rename(columns={'sum': 'Total_Spent', 'mean': 'Avg_Order', 'count': 'Num_Orders'})
print(cust_stats)


[GROUPBY] Total Sales by Region:
Region
North     358
South    2615
West     2275
Name: Amount, dtype: int64

[GROUPBY] Sales by Region AND Segment:
  Region   Segment  Amount
0  North  Standard     358
1  South   Premium     149
2  South  Standard    2466
3   West  Standard    2275

[GROUPBY] Custom Aggregations (Sum, Mean, Count):
         Total_Spent  Avg_Order  Num_Orders
Name                                       
Bob              358     358.00           1
Charlie         2466     616.50           4
David            149     149.00           1
Eve             2275     568.75           4
